In [ ]:

# Desinstalar versiones problemáticas previas
!pip uninstall -y torch torchvision torchaudio jax jaxlib tb-nightly tensorboard numpy

# Reinstalar NumPy compatible (por conflictos con compilación de módulos)
!pip install numpy==1.26.4

# Instalar versiones compatibles de PyTorch con CUDA 11.8
!pip install torch==2.0.0+cu118 torchvision==0.15.0+cu118 torchaudio==2.0.0+cu118 --index-url https://download.pytorch.org/whl/cu118

# Instalar YOLOv8 versión estable
!pip install ultralytics==8.0.100

# Instalar y actualizar Weights & Biases
!pip install --upgrade wandb


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: jax 0.5.2
Uninstalling jax-0.5.2:
  Successfully uninstalled jax-0.5.2
Found existing installation: jaxlib 0.5.1
Uninstalling jaxlib-0.5.1:
  Successfully uninstalled jaxlib-0.5.1
Found existing installation: tensorboard 2.18.0
Uninstalling tensorboard-2.18.0:
  Successfully uninstalled tensorboard-2.18.0
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 96.7 MB/s eta

In [ ]:
# ✅ Login a W&B
import wandb
wandb.login(key="854dc4c683777e6819e6cb34b559be45c1af4435")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lizarragaiker1 (lizarragaiker1-u-tad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import os
import time
import shutil
import numpy as np
import torch
import wandb
from ultralytics import YOLO

def train():
    with wandb.init() as run:
        config = wandb.config

        model_save_dir = '/content/drive/MyDrive/TFG/modelos_guardados_final'
        os.makedirs(model_save_dir, exist_ok=True)

        model = YOLO('yolov8l.pt')

        inicio_entrenamiento = time.time()

        model.train(
            data='/content/drive/MyDrive/TFG/dataset_deteccion/data.yaml',
            epochs=config.epochs,
            batch=config.batch_size,
            lr0=config.lr0,
            optimizer=config.optimizer,
            weight_decay=config.weight_decay,
            cos_lr=True,
            patience=config.patience,
            imgsz=640,
            name='sweep_sin_aug'
        )

        fin_entrenamiento = time.time()
        tiempo_entrenamiento = fin_entrenamiento - inicio_entrenamiento

        gpu_memoria = torch.cuda.max_memory_allocated() / (1024**3)
        tamaño_modelo = os.path.getsize(os.path.join(model.trainer.save_dir, 'weights', 'best.pt')) / (1024**2)
        RUC = gpu_memoria * tiempo_entrenamiento + tamaño_modelo

        resultados = model.val(split='test', imgsz=640)
        precision = resultados.results_dict['metrics/precision(B)']
        recall = resultados.results_dict['metrics/recall(B)']
        map50 = resultados.results_dict['metrics/mAP50(B)']
        map5095 = resultados.results_dict['metrics/mAP50-95(B)']
        f1_score = (2 * precision * recall) / (precision + recall + 1e-16)
        OPS = (precision + recall + f1_score + map50 + map5095) / 5

        tiempos_inferencia = []
        test_images_path = '/content/drive/MyDrive/TFG/dataset_deteccion/images/test'
        test_images = [os.path.join(test_images_path, img) for img in os.listdir(test_images_path)]

        for img in test_images[:20]:
            t0 = time.time()
            model.predict(img, imgsz=640, verbose=False)
            tiempos_inferencia.append(time.time() - t0)

        AIT = np.mean(tiempos_inferencia)

        confusion_matrix = resultados.confusion_matrix.matrix
        num_classes = confusion_matrix.shape[0]
        class_names = ["Normal", "Adenoma", "Serrado"][:num_classes]

        wandb.log({
            "confusion_matrix": wandb.plot.confusion_matrix(
                probs=None,
                y_true=np.argmax(confusion_matrix, axis=1),
                preds=np.argmax(confusion_matrix, axis=0),
                class_names=class_names
            ),
            'Precision': precision,
            'Recall': recall,
            'f1_score': f1_score,
            'mAP50': map50,
            'mAP50_95': map5095,
            'OPS': OPS,
            'AIT': AIT,
            'RUC': RUC,
            'GPU_Memory_GB': gpu_memoria,
            'Training_Time_sec': tiempo_entrenamiento,
            'Model_Size_MB': tamaño_modelo
        })

        best_weight_src = os.path.join(model.trainer.save_dir, 'weights', 'best.pt')
        SAVE_MODEL_PATH = os.path.join(model_save_dir, f"modelo_{run.name}.pt")

        if os.path.exists(best_weight_src):
            shutil.copy(best_weight_src, SAVE_MODEL_PATH)
            wandb.save(SAVE_MODEL_PATH)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Crear sweep y ejecutarlo
import yaml

with open('/content/drive/MyDrive/TFG/dataset_deteccion/sweep.yaml') as file:
    sweep_config = yaml.safe_load(file)

sweep_id = wandb.sweep(sweep_config, project='YOLOv8-TFG-definitivo')
wandb.agent(sweep_id, function=train, count=10, mosaic=0.0)


Create sweep with ID: uh0bu4a4
Sweep URL: https://wandb.ai/lizarragaiker1-u-tad/YOLOv8-TFG-definitivo/sweeps/uh0bu4a4


wandb: Agent Starting Run: d0lns82i with config:
wandb: 	batch_size: 32
wandb: 	epochs: 100
wandb: 	lr0: 0.0055
wandb: 	optimizer: AdamW
wandb: 	patience: 20
wandb: 	weight_decay: 0.0008


New https://pypi.org/project/ultralytics/8.3.95 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.100 🚀 Python-3.11.11 torch-2.0.0+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/TFG/dataset_deteccion/data.yaml, epochs=100, patience=20, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=sweep_sin_aug, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_n

AIT,▁
GPU_Memory_GB,▁
Model_Size_MB,▁
OPS,▁
Precision,▁
RUC,▁
Recall,▁
Training_Time_sec,▁
f1_score,▁
lr/pg0,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▆█████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▂▁▁▁▁▁▁


wandb: Agent Starting Run: vpzlqk0l with config:
wandb: 	batch_size: 48
wandb: 	epochs: 100
wandb: 	lr0: 0.004
wandb: 	optimizer: SGD
wandb: 	patience: 20
wandb: 	weight_decay: 0.0012


...

wandb: Agent Starting Run: sqlsjd9a with config:
wandb: 	batch_size: 64
wandb: 	epochs: 100
wandb: 	lr0: 0.004
wandb: 	optimizer: AdamW
wandb: 	patience: 20
wandb: 	weight_decay: 0.0012


New https://pypi.org/project/ultralytics/8.3.95 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.100 🚀 Python-3.11.11 torch-2.0.0+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/TFG/dataset_deteccion/data.yaml, epochs=100, patience=20, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=sweep_sin_aug, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_n

AIT,▁
GPU_Memory_GB,▁
Model_Size_MB,▁
OPS,▁
Precision,▁
RUC,▁
Recall,▁
Training_Time_sec,▁
f1_score,▁
lr/pg0,█▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▃████████▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: e1o59rn2 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 100
wandb: 	lr0: 0.0048
wandb: 	optimizer: SGD
wandb: 	patience: 20
wandb: 	weight_decay: 0.0012


New https://pypi.org/project/ultralytics/8.3.96 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.100 🚀 Python-3.11.11 torch-2.0.0+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/TFG/dataset_deteccion/data.yaml, epochs=100, patience=20, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=sweep_sin_aug, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms